In [23]:
from fbprophet import Prophet
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import random
import multiprocessing
import threading
from multiprocessing import Pool, freeze_support
from multiprocessing import Manager
import itertools
import math
plt.rcParams['figure.figsize']=(20,10)
plt.style.use('ggplot')

In [24]:
start=time.time()
#Read the data file
df=pd.read_excel(r'KPI Analysis Result_Query_Result_20210503151726224.xlsx')
df=df.iloc[0:2292,:]
#df=df.iloc[65782:68086,:]
#df=df.iloc[0:3332,:]

In [25]:
SiteName_array=[]
SiteId_array=[]
RAT_array=[]
CellId_array=[]

#Add the columns 'Site Name','Site Id','RAT' and 'Cell Id'
#for i in range(65782,68086):
for i in range(0,len(df)):
    
    SiteName = (df['Cell Name'][i].split('-'))[0]
    SiteName_array.append(SiteName)
    
    SiteId = (df['Cell Name'][i].split('-'))[1]
    SiteId_array.append(SiteId)
    
    RAT_CellId = (df['Cell Name'][i].split('-'))[2]
    RAT = (RAT_CellId.split('_'))[0]
    RAT_array.append(RAT)
    CellId = (RAT_CellId.split('_'))[1]
    CellId_array.append(CellId)

df['Site Name']=SiteName_array
df['Site ID']= SiteId_array
df['RAT']=RAT_array
df['Cell Id']=CellId_array

In [26]:
#Define the 'Sector Id' column values to cell groups 1,2,3 and 4
SectorId_array=[]
#for i in range(65782,68086):
for i in range(0,len(df)):
    if (df['Cell Id'][i]=='A'or df['Cell Id'][i]=='J'or df['Cell Id'][i]=='P'or df['Cell Id'][i]=='U'):
        SectorId_array.append(1)
    elif (df['Cell Id'][i]=='B'or df['Cell Id'][i]=='K'or df['Cell Id'][i]=='Q'or df['Cell Id'][i]=='V'):
        SectorId_array.append(2)
    elif (df['Cell Id'][i]=='C'or df['Cell Id'][i]=='L'or df['Cell Id'][i]=='R'or df['Cell Id'][i]=='W'):
        SectorId_array.append(3)
    elif (df['Cell Id'][i]=='D'or df['Cell Id'][i]=='M'or df['Cell Id'][i]=='S'or df['Cell Id'][i]=='X'):
        SectorId_array.append(4)

df['Sector Id'] = SectorId_array

In [27]:
#Define the 'Sector Unique Id' column
df['Sector Unique Id'] = df['Site ID']+'_'+df['Sector Id'].map(str)

In [28]:
#global result_df
result_df = pd.DataFrame()
#global avg_result_df
avg_result_df = pd.DataFrame()
drop_df = pd.DataFrame()

In [29]:
def cal_pred(j):
    t1=time.time()
    global result_df
    global avg_result_df
    global drop_df
    #dfs_list = Manager().list()
    #global result_final_df
    #Get the data frames as groups according to the 'Sector Unique Id'
    grouped = df.groupby(df['Sector Unique Id'])
    array=[]
    #Get the 'Sector Unique Id' values into an array
    array=df['Sector Unique Id'].unique()
    #Get the data frames as groups according to the 'Sector Unique Id'
    g=grouped.get_group(array[j])
    sec_uni_id=array[j]
    #Group the data frame according to the 'Date'
    gr_date = g.groupby(['Date']).agg({'Sector Unique Id':'first','Ish.L.TrafficVolume.DL.GB(GB)': 'sum'}).reset_index()
    print(gr_date)
    #Check whether the length of the dataframe is below 80% percent(240) of the total data points
    if (len(gr_date)<240):
        #increase the dropped cell count
        #drop_count+=1
        #When the length of the dataframe is below 80% percent(240) of the total data points, check whether the sector is a newly turned on sector or not
        if gr_date['Date'][len(gr_date)-1]==pd.Timestamp('2021-05-02 00:00:00') :
            #calculate the average network traffic volume values of last 5 days
            avg_traf_vol = gr_date['Ish.L.TrafficVolume.DL.GB(GB)'].iloc[len(gr_date)-5:len(gr_date)].mean(axis=0)
            #Create the dates for next 6 months
            date_array = pd.date_range(start='05/03/2021', periods=180, freq='D')
            #Apppend the average network traffic volume values for next 180 days
            for k in range(0,len(date_array)):
                avg_result_df = avg_result_df.append({'ds':date_array[k],'Sector Unique Id':sec_uni_id,'yhat_rescaled':avg_traf_vol}, ignore_index=True)
            
            print(sec_uni_id+' sector has the data from '+str(gr_date['Date'][0])+'. The sector has new cells and cannot make prediction due to less data.')
            #drop_df = drop_df.append({'ds':'Dropped','Sector Unique Id':sec_uni_id,'yhat_rescaled':'Sector is a new one and includes the data from'+str(gr_date['Date'][0])+'. Prediction cannot be done due to less data.'},ignore_index=True)
            #drop_df = drop_df.append({'Sector Unique Id':sec_uni_id,'Description':'Cells are new and includes the data from'+str(gr_date[0])+'. Prediction cannot be done due to less data.'},ignore_index=True)
            #drop_df.to_excel(drop_cells)
        else :
            print(sec_uni_id+' sector has '+str(len(gr_date))+'data points. Require to drop due to less data.')
            drop_df = drop_df.append({'ds':'Dropped','Sector Unique Id':sec_uni_id,'yhat_rescaled':'Many data points are missing. Prediction cannot be done.'},ignore_index=True)
            #drop_df = drop_df.append({'Sector Unique Id':sec_uni_id,'Description':'Many data points are missing. Prediction cannot be done.'},ignore_index=True)
            #drop_df.to_excel(drop_cells)
    #Identify the '0' value count is greater than the 20% of the total number of data points
    elif (gr_date['Ish.L.TrafficVolume.DL.GB(GB)']==0).astype(int).sum(axis=0)>(0.2*len(gr_date)):  #identify the rows having zeros as data values more than 80% of the dataframe
        print(sec_uni_id+' Sector has '+str((gr_date['Ish.L.TrafficVolume.DL.GB(GB)']==0).astype(int).sum(axis=0))+' zeros.Require to drop')
        drop_df = drop_df.append({'ds':'Dropped','Sector Unique Id':sec_uni_id,'yhat_rescaled':'Many data points have zeros. Prediction cannot be done.'},ignore_index=True)
        #drop_for_zero+=1
        #drop_df = drop_df.append({'Sector Unique Id':sec_uni_id,'Description':'Many data points have zeros. Prediction cannot be done.'},ignore_index=True)
        #drop_df.to_excel(drop_cells)
    else:
        dataset = gr_date.filter(['Date','Ish.L.TrafficVolume.DL.GB(GB)'])
        #Check the missing values in the data frame and put them into an array
        zero_index_array=[]
        
        for i in range(0,len(dataset)):
            if (dataset.iloc[i]['Ish.L.TrafficVolume.DL.GB(GB)']==0):
                zero_index_array.append(i)
                
        if len(zero_index_array)>0 :
            for k in zero_index_array:
                #Check whether the first value in the data frame is zero and replece the zeros with the average of nearby 10 data points
                if 0 in zero_index_array :
                    dataset.replace(0,dataset.iloc[k+len(zero_index_array)+1:k+len(zero_index_array)+11].mean(axis=0),inplace=True)
                    break
                #Check whether the last value in the data frame is zero and replece the zeros with the average of nearby 10 data points
                elif (len(dataset)-1) in zero_index_array:
                    dataset.replace(0,dataset.iloc[k-len(zero_index_array)-9:k-len(zero_index_array)+1].mean(axis=0),inplace=True)
                    break
                #Replace the zeros by the average of nearby 10 data values
                else:
                    dataset_sub=(dataset.iloc[k-5:k+6])[(dataset.iloc[k-5:k+6])['Ish.L.TrafficVolume.DL.GB(GB)']!=0]
                    dataset.replace(0,dataset_sub.mean(axis=0),inplace=True)
        #dataset.replace(0,dataset.mean(axis=0),inplace=True)
        dataset.columns = ['ds', 'y']
        dataset['ds']= pd.to_datetime(dataset['ds'])
        #Make a column to hold the original network traffic volume values
        dataset['y_orig']=dataset.y
        #Take the log scale of the data values in order to reduce the skewness
        dataset['y'] = np.log(dataset['y'])
        #acc_train_tot=0
        #r2_train_tot=0
        #Take the length of initial 80% of the data frame as the initial training data set length
        train_initial_len = math.ceil(len(dataset)*0.8)
        initial_dataset = dataset
        #For predict month by month create a for loop
        for k in range(6):
            new_dataset=pd.DataFrame()
            print(dataset)
            #Take the length of initial 80% of the data frame as the training data set length
            train_data_len = math.ceil(len(dataset)*0.8)
            #Take the length of test data set
            test_data_len =len(dataset)-train_data_len
            #Assign the training data set
            train =dataset.drop(dataset.index[-test_data_len:])
            #Parameters which are chosen to tune
            params_grid = {    
                'changepoint_prior_scale': [0.005,0.001,0.01,0.1,0.5],  
                'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0]
            }

            grid = ParameterGrid(params_grid)
            #Create a dataframe to store the 'MAPE' and 'Parameters'
            model_parameters = pd.DataFrame(columns = ['MAPE','Parameters'])
        
            for p in grid:
                
                random.seed(0)
                #Build the Prophet model for train
                train_model =Prophet(seasonality_prior_scale = p['seasonality_prior_scale'],
                             changepoint_prior_scale = p['changepoint_prior_scale'],
                             interval_width=0.95).add_seasonality(name='monthly',period=30.5,fourier_order=12).add_seasonality(name='dailly',period=1,fourier_order=15).add_seasonality(name='weekly',period=7,fourier_order=20).add_seasonality(name='quarterly',period=365.25/4,fourier_order=5) 
                #Fit the Prophet model
                train_model.fit(train)
                #Make the future data frame for predict
                train_forecast = train_model.make_future_dataframe(periods=test_data_len, freq='D',include_history = False)
                #Get the predictions
                train_forecast = train_model.predict(train_forecast)
                #Get the predicted values in the original scale
                train_forecast['yhat_rescaled'] = np.exp(train_forecast['yhat'])
                test=train_forecast[['ds','yhat_rescaled']]
                #Take the Mean Absolute Percentage Error
                MAPE = mean_absolute_percentage_error(dataset['y_orig'][-test_data_len:],abs(test['yhat_rescaled']))
                #Assign the values 'MAPE' values with parameters in to the data frame
                model_parameters = model_parameters.append({'MAPE':MAPE,'Parameters':p},ignore_index=True)
        
            #Sort the data frame according to the 'MAPE' value
            parameters = model_parameters.sort_values(by=['MAPE'])
            parameters = parameters.reset_index(drop=True)
            #Get the best parameters
            best_params = parameters['Parameters'][0]
            print('Best Parameters for'+sec_uni_id+':',best_params)
            #Build the Prophet model
            model = Prophet(interval_width=0.95, changepoint_prior_scale=best_params['changepoint_prior_scale'], seasonality_prior_scale = best_params['seasonality_prior_scale']).add_seasonality(name='monthly',period=30.5,fourier_order=12).add_seasonality(name='dailly',period=1,fourier_order=15).add_seasonality(name='weekly',period=7,fourier_order=20).add_seasonality(name='quarterly',period=365.25/4,fourier_order=5)
            #Fit the model with the train data
            model.fit(train) 
            #Make the future data frame for test data set
            future = model.make_future_dataframe(periods=test_data_len)
            #Get the predicted values
            forecast = model.predict(future)
            #Build a matrix to hold the predicted values and original values
            metric_df = forecast.set_index('ds')[['yhat']].join(dataset.set_index('ds').y_orig).reset_index()
            #Get the predicted values in the original scale
            metric_df['yhat_rescaled'] = np.exp(metric_df['yhat'])
            print(metric_df)
            #Calculate the r2_score
            r2score=r2_score(metric_df['y_orig'], metric_df['yhat_rescaled'])
            print('r2_score =',r2score)
            #r2_train_tot+=r2score
            y_true = metric_df['y_orig'][-test_data_len:]
        
            y_pred = metric_df['yhat_rescaled'][-test_data_len:]
            #Get the mean absolute error 
            mae = mean_absolute_error(y_true, y_pred)
            print('MAE: %.3f' % mae)
            #Get the mean absolute percentage error 
            mape=np.mean(np.abs((y_true - y_pred) / y_true)) * 100
            print('Mean Absolute Percentage Error (MAPE) =', mape)
            #Get the accuracy
            Acc= 100-mape
            print('Accuracy of the prediction=',Acc) 
            #Take the sum of the accutracy values in a sector data frame
            #acc_train_tot+=Acc
            #Make the future data frame for next 6 months' dates
            future_6 = model.make_future_dataframe(periods=(test_data_len+30))
            #Get the predictions
            forecast_6 = model.predict(future_6)
            
            metric_df_2 = forecast_6.set_index('ds')[['yhat', 'yhat_lower','yhat_upper']].join(dataset.set_index('ds').y_orig).reset_index()
            #Get the predicted values in the original scale
            metric_df_2['yhat_rescaled'] = np.exp(metric_df_2['yhat'])
            #Put the 'Sector Unique Id' in to a column in the matrix
            for i in range (0,len(g)):
                metric_df_2['Sector Unique Id'] =sec_uni_id
            
            
        #metric_df_2.set_index('ds', inplace=True)
        
            print(metric_df_2[['yhat_rescaled','y_orig']])
        
            #model.plot(forecast_6);
            #plt.savefig('Prediction_Sector_Vise_Append/PredFinalVisu_Append/'+sec_uni_id+'_next_6_month_plot1.png')
        
            #model.plot_components(forecast_6);
            #plt.savefig('Prediction_Sector_Vise_Append/PredFinalVisu_Append/'+sec_uni_id+'_next_6_month_plot2.png') 
            #plt.show()
       
            #Append the results with the training data set
            new_dataset=new_dataset.append(metric_df_2[['ds','yhat','yhat_rescaled']][-30:])
            new_dataset.columns=['ds','y','y_orig']
            print(new_dataset)
        
            dataset = dataset.append(new_dataset, ignore_index=True)
            print(dataset)
            
        #Average training accuracy for the sector
        #avg_train_acc=acc_train_tot/6
        #print('Average accuracy:',avg_train_acc)
        #Average r2_score for the sector
        #avg_train_r2=r2_train_tot/6
        #print('Average r2_score :',avg_train_r2)
        #Sum of the r2_scores of sectors
        #r2_tot+=avg_train_r2
        #Sum of the accuracies of sectors
        #acc_tot+=avg_train_acc
        
        #Set the 'ds' column as the index column
        metric_df_2.set_index('ds', inplace=True)
        #metric_df_2[['yhat_rescaled','y_orig']][train_initial_len:].to_excel('Prediction_Sector_Vise_Append/PredFinalData2/'+sec_uni_id+'.xlsx')
        #metric_df_2[['yhat_rescaled']][train_initial_len:].to_excel('Prediction_Sector_Vise_Append/PredFinalData/'+sec_uni_id+'.xlsx')
        #Results of the prediction were append to the data frame
        result_df = result_df.append(metric_df_2[['Sector Unique Id','yhat_rescaled']][train_initial_len:])
        ##########result_df = result_df.append({'Date':date_array[j],'Cell Name':name,'Predicted Traffic Volume':metric_df_2[['yhat_rescaled']]},ignore_index=True)
        #result_df.to_excel(predictions_6months)
        
        #Plot of 'Actual Value' and 'Predicted Value'
        fig, ax1 = plt.subplots()
        ax1.plot(metric_df_2.y_orig,color='b',label='Actual Value')
        ax1.plot(metric_df_2.yhat_rescaled,color='r',label='Predicted Value')
        ax1.fill_between(metric_df_2.index, np.exp(metric_df_2['yhat_upper']), np.exp(metric_df_2['yhat_lower']), alpha=0.5, color='lightblue')
        ax1.set_ylabel('Network Traffic Volume')
        ax1.set_xlabel('Date')
        ax1.legend() #get the legend
        plt.savefig('Prediction_Sector_Vise_Append/PredFinalvisu_6 month/'+sec_uni_id+'__6_month_orig_plot1.png')
        plt.show()
        
        #Plot of train data, test data and predicted values
        plt.plot(metric_df_2['y_orig'][0:train_initial_len], label='Training',color='g')
        plt.plot(metric_df_2['y_orig'][train_initial_len:len(initial_dataset)], label='Test',color='r')
        plt.plot(metric_df_2['yhat_rescaled'][train_initial_len:], label='Predicted',color='b')
        plt.legend()
        plt.savefig('Prediction_Sector_Vise_Append/PredFinalTrainTestPred_6_month/'+sec_uni_id+'_plot1.png')
        plt.show()
        #Increase the number of predicted sectors
        #count+=1
        t2=time.time()
        print('Time for '+sec_uni_id+' cell :',(t2-t1))

 
    if len(avg_result_df)>0:
        #Set the index column of the 'avg_result_df' data frame
        avg_result_df=avg_result_df.set_index('ds')
        #Combine the two data frames having the predictions
        result_df=pd.concat([result_df,avg_result_df],axis=0)
    if len(drop_df)>0:
        #Set the index column of the 'avg_result_df' data frame
        drop_df=drop_df.set_index('ds')
        #Combine the two data frames having the predictions
        result_df=pd.concat([result_df,drop_df],axis=0) 
        
    return result_df

In [30]:
#Define the multiprocessing method
def run_multiprocessing(func, tasks, n_processors):
    with Pool(processes=n_processors) as pool:
        #return pool.starmap(func, iterable=tasks)
        result_final_df = pool.map(func, tasks)
        
        print(pd.concat(result_final_df,axis=0))

        return pd.concat(result_final_df,axis=0)

In [31]:
def main():
    array=[]
    #Get the 'Sector Unique Id' values into an array
    array=df['Sector Unique Id'].unique()
    result_final_df = pd.DataFrame()
    #global drop_df
    #drop_df = pd.DataFrame(columns = ['Sector Unique Id','Description'])
    avg_result_df_final=pd.DataFrame()
    drop_df = pd.DataFrame()
    n_processors=multiprocessing.cpu_count()
    x_ls = list(range(0,len(array)))
    out = run_multiprocessing(cal_pred, x_ls, n_processors)
     
    predictions_6months = pd.ExcelWriter('Prediction_Sector_Vise_Append/TrafficVolumePredictions_Direct_threding.xlsx', engine='xlsxwriter')
    #drop_cells = pd.ExcelWriter('Prediction_Sector_Vise_Append/DetailsOfDropCells_Direct_threading.xlsx', engine='xlsxwriter')
    #Write the results into the excel file
    out.to_excel(predictions_6months)

    predictions_6months.save()
    #drop_cells.save()

In [ ]:
if __name__ == "__main__":
    freeze_support()   # required to use multiprocessing
    main()

In [33]:
end=time.time()
print('Total Time: ',(end-start)) 

Total Time:  830.6626844406128
